In [81]:
import xml.etree.ElementTree as ET
import re

def preprocess_article(filename):
    
    # Get the text part of the article xml by using Python's XML parser
    tree = ET.parse(filename) # parse the xml tree
    text = tree.getroot()[1][3][7].text # get the contents of the text-node
    
    string = ""
    for c in text:
        string += c # convert the list of characters to a Python string
    
    # Clean the string
    string = re.sub(r'\n', ' ', string) # remove new-line
    string = re.sub(r'\{\{.*?\}\}', ' ', string) # remove any {{...}} sections (metadata)
    string = re.sub(r'\<.*?\>', ' ', string) # remove HTML tags
    string = re.sub(r'\[\[([a-zA-Z\d])*\|', ' ', string) # remove the target of wiki-markup links to other articles
    string = re.sub(r'\=.*?\=', ' ', string) # removes =...=
    string = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', ' ', string) # remove urls  
    string = string.lower() # lower case everything
    string = re.sub(r'\(.*?\)', ' ', string) # removes parenthses
    string = re.sub(r'[^a-z\d\s].*?', '', string) # remove any remaining non-alphanumeric characters
    string = re.sub(' +', ' ', string) # remove trailing spaces
    string = re.sub('^ ', '', string) # remove any space at the front of the string
    
    return string

def hash_tree(string, article, tree = {}):
    words = [(m.group(0), m.start()) for m in re.finditer(r'\S+', string)]
    for word in words:
        add_word_to_tree(tree, word[0], word[1], article)
    
    return tree
    
def add_word_to_tree(tree, word, index, article):
    
    if (len(word) < 1): print('Error, attempt to add word shorter than 1 in length')
    if (len(tree) < 1 or word[0] not in tree.keys()):
        tree[word[0]] = {} # add first letter of word to tree if not present
    if (len(word) == 1): # terminate recursion
        if ('nodes' not in tree[word].keys()):
            tree[word]['nodes'] = {article : []}
        elif (article not in tree[word]['nodes'].keys()):
            tree[word]['nodes'][article] =  []
        tree[word]['nodes'][article].append(index)
    else:
        add_word_to_tree(tree[word[0]], word[1:], index, article)

def search(s, tree):
    if (len(s) < 1):
        print('Invalid search string')
        return {}
    if (len(s) == 1):
        return tree[s]['nodes']
    else:
        return {} if s[0] not in tree.keys() else search(s[1:], tree[s[0]])


In [85]:

cat_string = preprocess_article('articles/Cat')
dog_string = preprocess_article('articles/Dog')
tree = {}
tree = hash_tree(cat_string, 'Cat', tree)
tree = hash_tree(dog_string, 'Dog', tree)

search('mammal', tree)


{'Cat': [56, 19595, 33098], 'Dog': [5039, 5849, 23972, 29811]}

In [77]:
cat_string

'the domestic cat is a small typically furry carnivorous mammal they are often called house cats when kept as indoor pets or simply cats when there is no need to distinguish them from other felids and felines cats are often valued by humans for companionship and for their ability to hunt vermin there are more than 70 cat breeds though different associations proclaim different numbers according to their standards cats are similar in cat anatomyanatomy to the other felids with a strong flexible body quick reflexes sharp retractable claws and teeth adapted to killing small prey cat senses fit a crepuscular and predatory ecological niche cats can hear sounds too faint or too high in frequency for human ears such as those made by mice and other small animals they can see in near darkness like most other mammals cats have poorer color vision and a better sense of smell than humans cats despite being solitary hunters are a social animalsocial species and cat communication includes the use of 